In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
import os

import torchvision
from torch import nn
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets
from torchvision.transforms import v2
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast


import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

from tqdm.notebook import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import optuna
from optuna.trial import TrialState

In [55]:
def define_model(trial):
    n_layersC = trial.suggest_int("n_layersC", 1, 3)
    image_size = 600
    layers = []
    in_features = 3

    for i in range(n_layersC):
        out_features = trial.suggest_int("n_neurons{}".format(i), 4, 128)
        layers.append(nn.Conv2d(in_channels=in_features, out_channels=out_features, kernel_size=5, stride=3, padding=2))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        in_features = out_features

        image_size = (image_size + 2*2 - 5) // 3 + 1

        image_size = (image_size - 2) // 2 + 1

    layers.append(nn.Flatten())

    num_features = in_features * image_size * image_size

    n_layers = trial.suggest_int("n_layers", 1, 3)
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 500)
        layers.append(nn.Linear(num_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        num_features = out_features

    layers.append(nn.Linear(num_features, 1))

    return nn.Sequential(*layers)

In [6]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [7]:
transform = v2.Compose([
      v2.ToTensor(),
      v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform2 = v2.Compose([
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(90),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainset1 = datasets.ImageFolder(root="/content/drive/MyDrive/курсовая 2 курс /data/training",
                                        transform=transform)
trainset2 = datasets.ImageFolder(root="/content/drive/MyDrive/курсовая 2 курс /data/training",
                                        transform=transform2)
train_set = torch.utils.data.ConcatDataset([trainset1, trainset2])

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 16,
                                          shuffle=True, num_workers=2, drop_last = True)

validset = datasets.ImageFolder(root="/content/drive/MyDrive/курсовая 2 курс /data/validation",
                                      transform=transform)
valid_loader = torch.utils.data.DataLoader(validset, batch_size = 16,
                                          shuffle=False, num_workers=2, drop_last = True)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

criterion = nn.BCEWithLogitsLoss()
total_step = len(train_loader)
num_epochs = 10

cuda


In [45]:
def objective(trial):

    model = define_model(trial).to(device)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loss_list = []
    train_acc_list = []
    val_acc_list = []
    final_acc = []
    for epoch in tqdm(range(num_epochs)):
            # training
            for i, (images, labels) in enumerate(tqdm(train_loader)):
                if device.type == 'cuda':
                    images = images.cuda()
                    labels = labels.cuda()
                # feed forward
                outputs = model(images)
                labels = labels.unsqueeze(1).float()
                loss = criterion(outputs, labels)

                # back propagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # accuracy
                total = labels.size(0)
                predicted = (outputs.data > 0.5).float()
                correct = (predicted == labels).sum().item()
                if i % 16 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                          .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                                  (correct / total) * 100))
                    train_loss_list.append(loss.item())
                    train_acc_list.append(correct / total)

            # validation
            correct_count = 0
            total = 0
            for i, (images, labels) in enumerate(tqdm(valid_loader)):
                if device.type == 'cuda':
                    images = images.cuda()
                    labels = labels.cuda()
                # feed forward
                outputs = model(images)
                labels = labels.unsqueeze(1).float()

                predicted = (outputs.data > 0.5).float()
                correct = (predicted == labels).sum().item()
                total += labels.size(0)
                correct_count += correct

            accuracy = correct_count / total

            trial.report(accuracy, epoch)
            print(f'Validated accuracy {accuracy}')
            if trial.should_prune():
                print('PRUNED')
                raise optuna.exceptions.TrialPruned()
    return accuracy

In [57]:
study = optuna.create_study()
study.optimize(objective, n_trials = 50, timeout = 600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-01-19 17:11:54,086] A new study created in memory with name: no-name-1c0bae65-07eb-47c6-97d6-d0adb96a8f76


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/230 [00:00<?, ?it/s]

Epoch [1/10], Step [1/230], Loss: 0.6965, Accuracy: 68.75%
Epoch [1/10], Step [17/230], Loss: 0.7049, Accuracy: 43.75%
Epoch [1/10], Step [33/230], Loss: 0.7316, Accuracy: 56.25%
Epoch [1/10], Step [49/230], Loss: 0.7038, Accuracy: 31.25%
Epoch [1/10], Step [65/230], Loss: 0.6771, Accuracy: 56.25%
Epoch [1/10], Step [81/230], Loss: 0.6828, Accuracy: 43.75%
Epoch [1/10], Step [97/230], Loss: 0.7684, Accuracy: 62.50%
Epoch [1/10], Step [113/230], Loss: 0.6826, Accuracy: 37.50%
Epoch [1/10], Step [129/230], Loss: 0.6709, Accuracy: 56.25%
Epoch [1/10], Step [145/230], Loss: 0.6563, Accuracy: 43.75%
Epoch [1/10], Step [161/230], Loss: 0.6284, Accuracy: 31.25%
Epoch [1/10], Step [177/230], Loss: 0.7314, Accuracy: 50.00%
Epoch [1/10], Step [193/230], Loss: 0.6825, Accuracy: 56.25%
Epoch [1/10], Step [209/230], Loss: 0.6890, Accuracy: 56.25%
Epoch [1/10], Step [225/230], Loss: 0.6362, Accuracy: 37.50%


  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/230 [00:00<?, ?it/s]

Epoch [10/10], Step [1/230], Loss: 0.6191, Accuracy: 62.50%
Epoch [10/10], Step [17/230], Loss: 0.6243, Accuracy: 62.50%
Epoch [10/10], Step [33/230], Loss: 0.3908, Accuracy: 75.00%
Epoch [10/10], Step [49/230], Loss: 0.6076, Accuracy: 75.00%
Epoch [10/10], Step [65/230], Loss: 0.7357, Accuracy: 75.00%
Epoch [10/10], Step [81/230], Loss: 0.4411, Accuracy: 81.25%
Epoch [10/10], Step [97/230], Loss: 0.4905, Accuracy: 56.25%
Epoch [10/10], Step [113/230], Loss: 0.4944, Accuracy: 56.25%
Epoch [10/10], Step [129/230], Loss: 0.4016, Accuracy: 75.00%
Epoch [10/10], Step [145/230], Loss: 0.7635, Accuracy: 62.50%
Epoch [10/10], Step [161/230], Loss: 0.5765, Accuracy: 62.50%
Epoch [10/10], Step [177/230], Loss: 0.5809, Accuracy: 68.75%
Epoch [10/10], Step [193/230], Loss: 0.6559, Accuracy: 50.00%
Epoch [10/10], Step [209/230], Loss: 0.5592, Accuracy: 68.75%
Epoch [10/10], Step [225/230], Loss: 0.6427, Accuracy: 56.25%


  0%|          | 0/12 [00:00<?, ?it/s]

[I 2024-01-19 17:23:51,405] Trial 0 finished with value: 0.6197916666666666 and parameters: {'n_layersC': 2, 'n_neurons0': 8, 'n_neurons1': 53, 'n_layers': 1, 'n_units_l0': 306, 'dropout_0': 0.3864602773664628, 'optimizer': 'Adam', 'lr': 0.00011186579359733082}. Best is trial 0 with value: 0.6197916666666666.


Validated accuracy 0.6197916666666666
Study statistics: 
  Number of finished trials:  1
  Number of pruned trials:  0
  Number of complete trials:  1
Best trial:
  Value:  0.6197916666666666
  Params: 
    n_layersC: 2
    n_neurons0: 8
    n_neurons1: 53
    n_layers: 1
    n_units_l0: 306
    dropout_0: 0.3864602773664628
    optimizer: Adam
    lr: 0.00011186579359733082
